In [6]:
import re, os
from pandas_summary import DataFrameSummary
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
!pip install pandas_summary scikit-learn


In [12]:
PATH = pathlib.Path(os.getcwd()) 

In [9]:
data_file = PATH/"data/bulldozzer.csv"
data_file, data_file.is_file()

(WindowsPath('C:/Users/Jisha Manjesh.DESKTOP-J86V7RQ/data/bulldozzer.csv'),
 False)

# Reading

In [13]:
df_raw = pd.read_csv(data_file, low_memory=False, parse_dates=["saledate"]) 
df_raw.head()

FileNotFoundError: [Errno 2] File b'C:\\Users\\Jisha Manjesh.DESKTOP-J86V7RQ\\data\\bulldozzer.csv' does not exist: b'C:\\Users\\Jisha Manjesh.DESKTOP-J86V7RQ\\data\\bulldozzer.csv'

In [16]:
df_raw.shape

(401125, 53)

In [17]:
401125 * 53

21259625

In [101]:
df_raw.dtypes

SalesID                       int64
SalePrice                   float64
MachineID                     int64
ModelID                       int64
datasource                    int64
auctioneerID                float64
YearMade                      int64
MachineHoursCurrentMeter    float64
UsageBand                    object
fiModelDesc                  object
fiBaseModel                  object
fiSecondaryDesc              object
fiModelSeries                object
fiModelDescriptor            object
ProductSize                  object
fiProductClassDesc           object
state                        object
ProductGroup                 object
ProductGroupDesc             object
Drive_System                 object
Enclosure                    object
Forks                        object
Pad_Type                     object
Ride_Control                 object
Stick                        object
Transmission                 object
Turbocharged                 object
Blade_Extension             

In [11]:
df_raw.T

,0,1,2,3,4,5,6,7,8,9,...,401115,401116,401117,401118,401119,401120,401121,401122,401123,401124
SalesID,1139246,1139248,1139249,1139251,1139253,1139255,1139256,1139261,1139272,1139275,...,6333290,6333302,6333307,6333311,6333335,6333336,6333337,6333338,6333341,6333342
SalePrice,66000,57000,10000,38500,11000,26500,21000,27000,21500,65000,...,10000,8500,10000,9500,8500,10500,11000,11500,9000,7750
MachineID,999089,117657,434808,1026470,1057373,1001274,772701,902002,1036251,1016474,...,1843374,1825337,1821747,1828862,1798293,1840702,1830472,1887659,1903570,1926965
ModelID,3157,77,7009,332,17311,4605,1937,3539,36003,3883,...,21437,21437,21437,21437,21435,21439,21439,21439,21435,21435
datasource,121,121,121,121,121,121,121,121,121,121,...,149,149,149,149,149,149,149,149,149,149
auctioneerID,3,3,3,3,3,3,3,3,3,3,...,2,2,2,2,2,1,1,1,2,2
YearMade,2004,1996,2001,2001,2007,2004,1993,2001,2008,1000,...,2005,2005,2005,2006,2005,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,68,4640,2838,3486,722,508,11540,4883,302,20700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UsageBand,Low,Low,High,High,Medium,Low,High,High,Low,Medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00,2008-12-18 00:00:00,2004-08-26 00:00:00,2005-11-17 00:00:00,2009-08-27 00:00:00,2007-08-09 00:00:00,...,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [10]:
df_raw.shape

(401125, 53)


In any sort of data science work, it's important to look at your data, to make sure you understand the format, how it's stored, what type of values it holds, etc. Even if you've read descriptions about your data, the actual data **may not be what you expect**.

In [18]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [20]:
display_all(df_raw.tail())

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,30NX,30,NX,NaN,NaN,Mini,"Hydraulic Excavator, Track - 2.0 to 3.0 Metric...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401124,6333342,7750,1926965,21435,149,2.0,2005,NaN,NaN,2011-10-25,30NX,30,NX,NaN,NaN,Mini,"Hydraulic Excavator, Track - 2.0 to 3.0 Metric...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [22]:
display_all(df_raw.describe().T)

,count,mean,std,min,25%,50%,75%,max
SalesID,401125.0,1.919713e+06,909021.492667,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0
SalePrice,401125.0,3.109971e+04,23036.898502,4750.0,14500.0,24000.0,40000.0,142000.0
MachineID,401125.0,1.217903e+06,440991.954249,0.0,1088697.0,1279490.0,1468067.0,2486330.0
ModelID,401125.0,6.889703e+03,6221.777842,28.0,3259.0,4604.0,8724.0,37198.0
datasource,401125.0,1.346658e+02,8.962237,121.0,132.0,132.0,136.0,172.0
auctioneerID,380989.0,6.556040e+00,16.976779,0.0,1.0,2.0,4.0,99.0
YearMade,401125.0,1.899157e+03,291.797469,1000.0,1985.0,1995.0,2000.0,2013.0
MachineHoursCurrentMeter,142765.0,3.457955e+03,27590.256413,0.0,0.0,0.0,3025.0,2483300.0


In [7]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Generally, selecting the metric(s) is an important part of the project setup. However, in this case Kaggle tells us what metric to use: RMSLE (root mean squared log error) between the actual and predicted auction prices

In [14]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

NameError: name 'df_raw' is not defined

In [22]:
display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,11.0974,10.9508,9.21034,10.5584,9.30565
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [23]:
df_raw.SalePrice

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
5         10.184900
6          9.952278
7         10.203592
8          9.975808
9         11.082143
10        10.085809
11        10.021271
12        10.491274
13        10.325482
14        10.239960
15         9.852194
16         9.510445
17         9.159047
18         9.433484
19         9.350102
20        10.621327
21        10.448715
22        10.165852
23        11.198215
24        10.404263
25         9.433484
26         9.648595
27        10.878047
28        10.736397
29        11.396392
            ...    
401095     9.259131
401096     9.210340
401097     9.047821
401098     9.259131
401099     9.305651
401100     9.259131
401101     9.210340
401102     9.259131
401103     9.433484
401104     9.259131
401105     9.210340
401106     9.259131
401107     9.433484
401108     9.259131
401109     9.259131
401110     9.259131
401111     9.210340
401112     9.259131
401113     9.259131


Why log ?


m = RandomForestRegressor(n_jobs=-1) 
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice) # y ~ x 

# Dates

In [23]:
df_raw.saledate

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

What are the intersting things in dat ?

In [16]:
fld = df_raw.saledate
fld

NameError: name 'df_raw' is not defined

In [17]:
dir(fld.dt)

NameError: name 'fld' is not defined

In [29]:
fld[0].week

46

In [31]:
fld[0].is_month_end

False

In [32]:
fld[0].year, fld[0].is_month_end

(2006, False)

In [48]:
"sale"+"year"

'saleyear'

In [38]:
re.sub('[Dd]ate$', '|', "DatesaleDate")

'Datesale|'

In [37]:
re.sub('^[Dd]ate', '|', "DatesaleDate")

'|saleDate'

In [57]:
fld[0].year

2006

In [62]:
getattr(fld.dt, "year")

0         2006
1         2004
2         2004
3         2011
4         2009
5         2008
6         2004
7         2005
8         2009
9         2007
10        2008
11        2006
12        2005
13        2006
14        2006
15        2006
16        2007
17        2010
18        2006
19        2005
20        2006
21        2006
22        2007
23        2006
24        2004
25        2006
26        2006
27        2007
28        2007
29        2006
          ... 
401095    2011
401096    2011
401097    2011
401098    2011
401099    2011
401100    2011
401101    2011
401102    2011
401103    2011
401104    2011
401105    2011
401106    2011
401107    2011
401108    2011
401109    2011
401110    2011
401111    2011
401112    2011
401113    2011
401114    2011
401115    2011
401116    2011
401117    2011
401118    2011
401119    2011
401120    2011
401121    2011
401122    2011
401123    2011
401124    2011
Name: saledate, Length: 401125, dtype: int64

In [39]:
d = "c"

In [49]:
"Year".lower() 

'year'

In [77]:
attr = [('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start')]
for n in attr:
    print(n.lower())
    x = getattr(fld.dt, n.lower())
    print(x)
    print("____________________________________")


year
0         2006
1         2004
2         2004
3         2011
4         2009
5         2008
6         2004
7         2005
8         2009
9         2007
10        2008
11        2006
12        2005
13        2006
14        2006
15        2006
16        2007
17        2010
18        2006
19        2005
20        2006
21        2006
22        2007
23        2006
24        2004
25        2006
26        2006
27        2007
28        2007
29        2006
          ... 
401095    2011
401096    2011
401097    2011
401098    2011
401099    2011
401100    2011
401101    2011
401102    2011
401103    2011
401104    2011
401105    2011
401106    2011
401107    2011
401108    2011
401109    2011
401110    2011
401111    2011
401112    2011
401113    2011
401114    2011
401115    2011
401116    2011
401117    2011
401118    2011
401119    2011
401120    2011
401121    2011
401122    2011
401123    2011
401124    2011
Name: saledate, Length: 401125, dtype: int64
___________________________________

In [41]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [42]:
"sale"+"Year"

'saleYear'

In [46]:
def add_datepart(df, fldname, drop=True):
    if not fldname in df_raw.columns :
        return 
    
    fld = df[fldname]

    # fldname = saledates
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    targ_pre = re.sub('[Dd]ate$', '', fldname) # sale
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())

    if drop: # dropped !!!
        df.drop(fldname, axis=1, inplace=True)

In [47]:
add_datepart(df_raw,"saledate")

In [49]:
"saledate" in df_raw.columns

False

In [50]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [92]:
df_raw.shape

(401125, 64)

In [79]:
df_raw.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,11.0974,10.9508,9.21034,10.5584,9.30565
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
fiModelDesc,521D,950FII,226,PC120-6E,S175


In [93]:
p = {1:2, 4:56}

In [95]:
for key, val in p.items():
    print(key, " :: ", val)

1  ::  2
4  ::  56


# Category 

In [103]:
df_raw["Pad_Type"]

0                         NaN
1                         NaN
2                         NaN
3                         NaN
4                         NaN
5         None or Unspecified
6                         NaN
7                  Reversible
8                         NaN
9                         NaN
10                     Street
11                        NaN
12                        NaN
13        None or Unspecified
14        None or Unspecified
15        None or Unspecified
16                        NaN
17                        NaN
18                        NaN
19        None or Unspecified
20                        NaN
21                        NaN
22                        NaN
23                        NaN
24        None or Unspecified
25                        NaN
26                        NaN
27                        NaN
28                        NaN
29                        NaN
                 ...         
401095                    NaN
401096                    NaN
401097    

In [107]:
display_all(df_raw.nunique())

SalesID                     401125
SalePrice                      899
MachineID                   341027
ModelID                       5218
datasource                       5
auctioneerID                    30
YearMade                        72
MachineHoursCurrentMeter     15152
UsageBand                        3
fiModelDesc                   4999
fiBaseModel                   1950
fiSecondaryDesc                175
fiModelSeries                  122
fiModelDescriptor              139
ProductSize                      6
fiProductClassDesc              74
state                           53
ProductGroup                     6
ProductGroupDesc                 6
Drive_System                     4
Enclosure                        6
Forks                            2
Pad_Type                         4
Ride_Control                     3
Stick                            2
Transmission                     8
Turbocharged                     2
Blade_Extension                  2
Blade_Width         

In [51]:
def to_cats(df):
    for col_name,col in df.items():
        if is_string_dtype(col): 
            df[col_name] = col.astype('category').cat.as_ordered()
            
to_cats(df_raw)

In [56]:
dir(df_raw["UsageBand"].cat)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_reset_cache',
 '_validate',
 'add_categories',
 'as_ordered',
 'as_unordered',
 'categorical',
 'categories',
 'codes',
 'index',
 'name',
 'ordered',
 'remove_categories',
 'remove_unused_categories',
 'rename_categories',
 'reorder_categories',
 'set_categories']

In [53]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [19]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

NameError: name 'df_raw' is not defined

In [57]:

getattr(df_raw["UsageBand"].cat ,"categories")

Index(['High', 'Low', 'Medium'], dtype='object')

In [112]:

getattr(df_raw["UsageBand"].cat ,"codes")

0         1
1         1
2         0
3         0
4         2
5         1
6         0
7         0
8         1
9         2
10        2
11        1
12       -1
13        2
14        1
15        2
16        1
17       -1
18        1
19        2
20        2
21        2
22        1
23        0
24        2
25        2
26        1
27       -1
28        1
29        2
         ..
401095   -1
401096   -1
401097   -1
401098   -1
401099   -1
401100   -1
401101   -1
401102   -1
401103   -1
401104   -1
401105   -1
401106   -1
401107   -1
401108   -1
401109   -1
401110   -1
401111   -1
401112   -1
401113   -1
401114   -1
401115   -1
401116   -1
401117   -1
401118   -1
401119   -1
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [21]:
df_raw.UsageBand.cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

In [65]:
df_raw.isnull().sum().sort_index()

Backhoe_Mounting     322453
Blade_Extension      375906
Blade_Type           321292
Blade_Width          375906
Coupler              187173
                      ...  
saleIs_year_start         0
saleMonth                 0
saleWeek                  0
saleYear                  0
state                     0
Length: 64, dtype: int64

In [68]:
len(df_raw)

401125

In [79]:
df_raw["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): [High < Low < Medium]

In [22]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [74]:
df_raw["Forks"].isnull().sum()

209048

!pip install -U feather-format

In [86]:
if 0:
    print("nithin")

In [87]:
def fix_missing(df, col,name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name + "_na"] = pd.isnull(col)
            df[name] = col.fillna(col.median())


def numericalize(df ,col,name ):
    if not is_numeric_dtype(col) :
        df[name] = pd.Categorical(col).codes+1
        
    


In [88]:

def proc_df(df , y_fld):
    df = df.copy()
    y = df[y_fld].values
    df.drop([y_fld], axis=1, inplace=True)
    for n,c in df.items():
        fix_missing(df,c,n)
        numericalize(df,c,n)
    return [df , y]

In [89]:
df, y =proc_df(df_raw, "SalePrice")

In [90]:
type(df_raw["SalePrice"].values)

numpy.ndarray

In [92]:
df_raw.shape

(401125, 64)

In [91]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')



ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

In [94]:
display_all(df.isnull().sum())

SalesID                        0
MachineID                      0
ModelID                        0
datasource                     0
auctioneerID                   0
YearMade                       0
MachineHoursCurrentMeter       0
UsageBand                      0
fiModelDesc                    0
fiBaseModel                    0
fiSecondaryDesc                0
fiModelSeries                  0
fiModelDescriptor              0
ProductSize                    0
fiProductClassDesc             0
state                          0
ProductGroup                   0
ProductGroupDesc               0
Drive_System                   0
Enclosure                      0
Forks                          0
Pad_Type                       0
Ride_Control                   0
Stick                          0
Transmission                   0
Turbocharged                   0
Blade_Extension                0
Blade_Width                    0
Enclosure_Type                 0
Engine_Horsepower              0
Hydraulics

In [27]:
display_all(df.tail().T)

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,0,0,0,0,0
UsageBand,0,0,0,0,0
fiModelDesc,657,657,657,483,483
fiBaseModel,207,207,207,159,159


In [95]:

m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)

/home/nithin/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [96]:
m.score(df,y)

0.981949182861473

In [ ]:
df , 